In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import ForeignKey
Base = declarative_base()
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Column, Integer, String, Float, MetaData
import pandas as pd

In [2]:
engine = create_engine("sqlite:///movie_rating_vs_revenue.sqlite", echo=False)
connection = engine.connect()
meta = MetaData()

In [3]:
# Need to use declarative base
Base = declarative_base()

In [4]:
#Movie class
class Movie(Base):
    __tablename__ = 'movie'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    content_rating = Column(String)
    year = Column(Integer)
    runtime = Column(Float)

In [5]:
#Financial Class
class Financial(Base):
    __tablename__ = 'financial'
    id = Column(Integer, ForeignKey("movie.id"), primary_key = True, nullable=False)
    budget = Column(Float)
    gross = Column(Float)
    profit = Column(Float)

In [6]:
#Review Info
class Reviews(Base):
    __tablename__ = 'reviews'
    id = Column(Integer, ForeignKey("movie.id"), primary_key = True, nullable=False)
    imdb_score = Column(Float)
    imdb_reviewers = Column(Integer)
    imdb_voters = Column(Integer)
    imdb_user_reviewers = Column(Integer)
    top_list_rating = Column(Float)
    top_list_votes = Column(Integer)

In [7]:
#Social Media Info
class SocialMedia(Base):
    __tablename__ = 'social_media'
    id = Column(Integer, ForeignKey("movie.id"), primary_key = True, nullable=False)
    movie_likes = Column(Integer)
    director_likes = Column(Integer)
    cast_likes = Column(Integer)
    actor1_likes = Column(Integer)
    actor2_likes = Column(Integer)
    actor3_likes = Column(Integer)

In [8]:
#Social Media Info
class Actor(Base):
    __tablename__ = 'actor'
    id = Column(Integer, primary_key = True, nullable=False)
    actor_name = Column(String)

In [9]:
#Actor-film Info
class Actor_Movie(Base):
    __tablename__ = 'actor_movie'
    id = Column(Integer, primary_key = True, nullable=False)
    actor_id = Column(Integer, ForeignKey("actor.id"), nullable=False)
    movie_id = Column(Integer, ForeignKey("movie.id"), nullable=False)

In [10]:
#Genre Info
class Genre(Base):
    __tablename__ = 'genre'
    id = Column(Integer, primary_key = True, nullable=False)
    genre_name = Column(String)

In [11]:
#Genre-film Info
class Genre_Movie(Base):
    __tablename__ = 'genre_movie'
    id = Column(Integer, primary_key = True, nullable=False)
    genre_id = Column(Integer, ForeignKey("actor.id"), nullable=False)
    movie_id = Column(Integer, ForeignKey("movie.id"), nullable=False)

In [12]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [13]:
# create a session
session = Session(engine)

In [14]:
session.commit()

In [15]:
meta.reflect(engine)
meta.tables.keys()

dict_keys(['actor', 'actor_movie', 'movie', 'financial', 'genre', 'genre_movie', 'reviews', 'social_media'])

In [16]:
all_df = pd.read_csv('combined_df.csv')

In [17]:
all_df.columns

Index(['title', 'content_rating', 'budget', 'gross', 'imdb_score',
       'num_critic_for_reviews', 'num_voted_users', 'num_user_for_reviews',
       'movie_facebook_likes', 'cast_total_facebook_likes',
       'actor_1_facebook_likes', 'actor_2_facebook_likes',
       'actor_3_facebook_likes', 'director_facebook_likes', 'genre', 'actors',
       'year', 'runtime_minutes', 'votes', 'metascore'],
      dtype='object')

In [18]:
movies_df = all_df[['title','content_rating','year','runtime_minutes']]
movies_df = movies_df.rename(columns = {'runtime_minutes':'runtime'})
movies_df.head()

,title,content_rating,year,runtime
0,Avatar,PG-13,2009,162
1,Pirates of the Caribbean: At World's End,PG-13,2007,169
2,Spectre,PG-13,2015,148
3,The Dark Knight Rises,PG-13,2012,164
4,John Carter,PG-13,2012,132


In [19]:
movies_df.to_sql('movie', connection, if_exists = 'append', index = False)

In [20]:
session.commit()

In [21]:
movie_check_df = pd.read_sql('SELECT * FROM movie', connection)

In [22]:
movie_check_df.head()

,id,title,content_rating,year,runtime
0,1,Avatar,PG-13,2009,162.0
1,2,Pirates of the Caribbean: At World's End,PG-13,2007,169.0
2,3,Spectre,PG-13,2015,148.0
3,4,The Dark Knight Rises,PG-13,2012,164.0
4,5,John Carter,PG-13,2012,132.0


In [23]:
financial_df = all_df[['budget','gross']].copy()
financial_df['profit'] = financial_df['gross'] - financial_df['budget']
financial_df.head()

,budget,gross,profit
0,237000000.0,760505847.0,523505847.0
1,300000000.0,309404152.0,9404152.0
2,245000000.0,200074175.0,-44925825.0
3,250000000.0,448130642.0,198130642.0
4,263700000.0,73058679.0,-190641321.0


In [24]:
financial_df.to_sql('financial', connection, if_exists = 'append', index = False)

In [25]:
session.commit()

In [26]:
financial_check_df = pd.read_sql('SELECT * FROM financial', connection)

In [27]:
financial_check_df.head()

,id,budget,gross,profit
0,1,237000000.0,760505847.0,523505847.0
1,2,300000000.0,309404152.0,9404152.0
2,3,245000000.0,200074175.0,-44925825.0
3,4,250000000.0,448130642.0,198130642.0
4,5,263700000.0,73058679.0,-190641321.0


In [32]:
review_df = all_df[['imdb_score','num_critic_for_reviews','num_voted_users','num_user_for_reviews','metascore','votes']]
review_df = review_df.rename(columns = {'num_critic_for_reviews':'imdb_reviewers','num_voted_users':'imdb_voters',
                              'num_user_for_reviews':'imdb_user_reviewers','metascore':'top_list_rating',
                              'votes':'top_list_votes'})
review_df.head()

,imdb_score,imdb_reviewers,imdb_voters,imdb_user_reviewers,top_list_rating,top_list_votes
0,7.9,723.0,886204,3054.0,83.0,935408
1,7.1,302.0,471220,1238.0,50.0,498821
2,6.8,602.0,275868,994.0,60.0,308981
3,8.5,813.0,1144337,2701.0,78.0,1222645
4,6.6,462.0,212204,738.0,51.0,220667


In [33]:
review_df.to_sql('reviews', connection, if_exists = 'append', index = False)

In [34]:
session.commit()

In [35]:
social_media_df = all_df[['movie_facebook_likes','director_facebook_likes','cast_total_facebook_likes',
                         'actor_1_facebook_likes','actor_2_facebook_likes','actor_3_facebook_likes']]
social_media_df.columns = ['movie_likes','director_likes','cast_likes','actor1_likes','actor2_likes','actor3_likes']
social_media_df.head()

,movie_likes,director_likes,cast_likes,actor1_likes,actor2_likes,actor3_likes
0,33000,0.0,4834,1000.0,936.0,855.0
1,0,563.0,48350,40000.0,5000.0,1000.0
2,85000,0.0,11700,11000.0,393.0,161.0
3,164000,22000.0,106759,27000.0,23000.0,23000.0
4,24000,475.0,1873,640.0,632.0,530.0


In [36]:
social_media_df.to_sql('social_media', connection, if_exists = 'append', index = False)

In [37]:
session.commit()

In [38]:
sm_check_df = pd.read_sql('SELECT * FROM social_media', connection)
sm_check_df.head()

,id,movie_likes,director_likes,cast_likes,actor1_likes,actor2_likes,actor3_likes
0,1,33000,0,4834,1000,936,855
1,2,0,563,48350,40000,5000,1000
2,3,85000,0,11700,11000,393,161
3,4,164000,22000,106759,27000,23000,23000
4,5,24000,475,1873,640,632,530


In [68]:
# Make a list of all the actors and their frequency of appearance
actor_df = all_df[['actors']]
actor_df = actor_df['actors'].str.split(',',-1,expand = True)
actor_df.head()

,0,1,2,3
0,Sam Worthington,Zoe Saldana,Sigourney Weaver,Michelle Rodriguez
1,Johnny Depp,Orlando Bloom,Keira Knightley,Geoffrey Rush
2,Daniel Craig,Christoph Waltz,Léa Seydoux,Ralph Fiennes
3,Christian Bale,Tom Hardy,Anne Hathaway,Gary Oldman
4,Taylor Kitsch,Lynn Collins,Willem Dafoe,Samantha Morton


In [69]:
 # To consolidate the lists, we're going to do a trick with value counts ... 
series1 = actor_df.iloc[:,0].value_counts()
series2 = actor_df.iloc[:,1].value_counts()
series3 = actor_df.iloc[:,2].value_counts()
series4 = actor_df.iloc[:,3].value_counts()

In [70]:
counts_df = pd.DataFrame({'0':series1, '1':series2, '2':series3, '3':series4}).reset_index()

In [71]:
counts_df.head()

,index,0,1,2,3
0,50 Cent,NaN,NaN,1.0,NaN
1,AJ Michalka,NaN,1.0,NaN,NaN
2,Aaron Burns,NaN,NaN,1.0,NaN
3,Aaron Eckhart,NaN,2.0,1.0,NaN
4,Aaron Yoo,NaN,NaN,NaN,1.0


In [72]:
actor_df = counts_df[['index']]
actor_df.columns = ['actor_name']
actor_df.head()

,actor_name
0,50 Cent
1,AJ Michalka
2,Aaron Burns
3,Aaron Eckhart
4,Aaron Yoo


In [62]:
actor_df.to_sql('actor', connection, if_exists = 'append', index = False)

In [63]:
genre_df = all_df[['genre']]
genre_df = genre_df['genre'].str.split(',',-1,expand = True)
genre_df.head()

,0,1,2
0,Action,Adventure,Fantasy
1,Action,Adventure,Fantasy
2,Action,Adventure,Thriller
3,Action,Thriller,None
4,Action,Adventure,Sci-Fi


In [64]:
 # To consolidate the lists, we're going to do a trick with value counts ... 
series1 = genre_df.iloc[:,0].value_counts()
series2 = genre_df.iloc[:,1].value_counts()
series3 = genre_df.iloc[:,2].value_counts()
count_df = pd.DataFrame({'0':series1, '1':series2, '2':series3}).reset_index()
genre_df = count_df[['index']]
genre_df.columns = ['genre_name']
genre_df.head()

,genre_name
0,Action
1,Adventure
2,Animation
3,Biography
4,Comedy


In [66]:
genre_df.to_sql('genre', connection, if_exists = 'append', index = False)

In [85]:
# To get the crosswalk between actors and movies, we'll start by regenerrating the spit actor list
actorlist_df = all_df[['actors']]
actorlist_df = actorlist_df['actors'].str.split(',',-1,expand = True)
actorlist_df.columns = ['actor_name','1', '2', '3']
actorlist_df.head()

,actor_name,1,2,3
0,Sam Worthington,Zoe Saldana,Sigourney Weaver,Michelle Rodriguez
1,Johnny Depp,Orlando Bloom,Keira Knightley,Geoffrey Rush
2,Daniel Craig,Christoph Waltz,Léa Seydoux,Ralph Fiennes
3,Christian Bale,Tom Hardy,Anne Hathaway,Gary Oldman
4,Taylor Kitsch,Lynn Collins,Willem Dafoe,Samantha Morton


In [87]:
actor_df = actor_df.reset_index()
actor_df.head()

,index,actor_name
0,0,50 Cent
1,1,AJ Michalka
2,2,Aaron Burns
3,3,Aaron Eckhart
4,4,Aaron Yoo


In [88]:
actorlist_df = actorlist_df.merge(actor_df,on = 'actor_name', how = 'inner')
actorlist_df.head()

,actor_name,1,2,3,index
0,Sam Worthington,Zoe Saldana,Sigourney Weaver,Michelle Rodriguez,1331
1,Sam Worthington,Liam Neeson,Rosamund Pike,Ralph Fiennes,1331
2,Sam Worthington,Liam Neeson,Ralph Fiennes,Jason Flemyng,1331
3,Sam Worthington,Liam Neeson,Ralph Fiennes,Jason Flemyng,1331
4,Sam Worthington,Elizabeth Banks,Jamie Bell,Mandy Gonzalez,1331
